In [1]:
from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader
from llama_index.storage.storage_context import StorageContext
from llama_index.llm_predictor import HuggingFaceLLMPredictor
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from IPython.display import Markdown, display
from llama_index import ServiceContext, LangchainEmbedding
import os
import pandas as pd

In [2]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [3]:
data_dir = "/home/jupyter/data_articles"
persist_dir = "/home/jupyter/persist"

In [4]:
article_df = pd.read_parquet("gs://scraped-news-article-data-null/april-2023.parquet")
article_df.head()

,source,id,category,title,published,body,summary,summary_type
0,cnbc,3717,Politics,DeSantis and allies ramp up Disney fight as mo...,2023-04-19T20:00:49+00:00,## In this article\nFollow your favorite stock...,* Florida Gov. Ron DeSantis ripped Disney over...,BULLETS
1,reuters,1996,LitigationCorporate StructureLawsuits,J&J talc unit faces skepticism on bankruptcy s...,2023-04-11T22:19:00,April 11 (Reuters) - A Johnson & Johnson (JNJ....,,NULL
2,reuters,10431,Aerospace & Defense,US GAO denies Lockheed protest of Textron $7 b...,2023-04-06T20:41:00,"WASHINGTON, April 6 (Reuters) - The U.S. Gover...",,NULL
3,reuters,14305,United States,"'Donald Trump raped me,' writer says at civil ...",2023-04-26T23:22:00,"NEW YORK, April 26 (Reuters) - A writer explai...",,NULL
4,reuters,10282,Charged,Walmart plans own EV charger network at U.S. s...,2023-04-06T16:52:00,"NEW YORK, April 6 (Reuters) - Walmart Inc (WMT...",,NULL


In [5]:
from pathlib import Path


for i in range(len(article_df.index)):
    with open(Path(data_dir, str(i) + ".txt"), "w") as fp:
        fp.write(article_df["body"].iloc[i])


In [6]:
import torch
from llama_index.prompts.prompts import SimpleInputPrompt


system_prompt = """<|SYSTEM|># StableLM Tuned (Alpha version)
- StableLM is a helpful and harmless open-source AI language model developed by StabilityAI.
- StableLM is excited to be able to help the user, but will refuse to do anything that could be considered harmful to the user.
- StableLM is more than just an information source, StableLM is also able to write poetry, short stories, and make jokes.
- StableLM will refuse to participate in anything that could harm a human.
""" 

# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

hf_predictor = HuggingFaceLLMPredictor(
    max_input_size=4096, 
    max_new_tokens=2048,
    temperature=0,
    do_sample=False,
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="stabilityai/stablelm-tuned-alpha-7b",
    model_name="stabilityai/stablelm-tuned-alpha-7b",
    device_map="auto",
    stopping_ids=[50278, 50279, 50277, 1, 0],
    tokenizer_kwargs={"max_length": 4096},
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16}
)
embed_model = LangchainEmbedding(HuggingFaceEmbeddings())
service_context = ServiceContext.from_defaults(chunk_size_limit=1024, llm_predictor=hf_predictor, embed_model=embed_model)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2
Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cuda
Use pytorch device: cuda


In [7]:
storage_context = StorageContext.from_defaults()
documents = SimpleDirectoryReader(data_dir).load_data()

In [8]:
new_index = GPTVectorStoreIndex.from_documents(documents, service_context=service_context, storage_context=storage_context)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 5325312 tokens
> [build_index_from_nodes] Total embedding token usage: 5325312 tokens


In [9]:
storage_context.persist(persist_dir=persist_dir)